# Определение среднего рейтинга лекарственного препарата. 

#### Цель: 
    По заданному отзыву, предсказать средний рейтинг препарата. 
    * Средний показатель рейтинга определяется следующим образом: вычисляется средняя сумма рейтинговых показателей по каждой категории (поля: rating_1, rating_2, rating_3, rating_4, rating_5), если полученное число <= 60, то метка класса отрицательная(0), если > 60, метка класса положительная (1).

#### Корпус: 
    https://cimm.kpfu.ru/seafile/f/cf558054df6c45f8aca5/
#### Команда: 
    Курнатовский Вадим, Петров Валерий

## Общее задание

#### Цель: 
    Реализовать классификатор на основе моделей машинного обучения, вывести наиболее значимые признаки.

#### Метрики качества: 
    - Точность (precision)
    - Полнота (recall)
    - F-мера (f-measure)

#### Задачи: 
1. Выделить из корпуса по 500 примеров на каждый класс, если классификация бинарная и по 200, если классов несколько.
2. Разбить примеры на обучающую и тестовую выборки в соотношении 80% к 20% соответственно. 
3. Реализовать классификатор на основе tf-idf и модели логистической регрессии, вывести метрики качества на тестовом множестве. Максимальное кол-во признаков: 50000, минимальная частота: 5.
4. Вывести наиболее значимые признаки (токены), используя один из указанных методов [3, 4].
5. Реализовать не менее 5-ти собственных признаков (можно больше), улучшающих результаты классификатора, полученные с использованием признака tf-idf. Оценить поочередно качество классификатора, обученного на tf-idf и каждом из признаков [5]. 
6. Оценить качество классификатора, обученного на tf-idf и всех реализованных признаках.
7. С помощью GridSearch вывести наиболее значимые признаки [4].
8. Запустить лучшую конфигурацию модели на полученных тестовых данных, предсказать результат. Отправить полученные результаты на проверку.
9. Вычистить и структурировать, загрузить его в git репозиторий.
10. Написать краткий отчет о проделанной работе и полученных результатах (см. прим. 2).

#### Задача № 0. Подготовка данных
1. Извлечение данных, относящихся к отзыву - полезная информация.
2. Нормализация извлеченной полезной информации.

In [10]:
import json
import re
from os.path import abspath

POS_RATING = 1
NEG_RATING = 0


def extract_payload(file_input, file_output, list_fields: list, int_number_items: int) -> None:
    i_item = 0
    for str_raw_review in file_input:
        try:
            dict_review = json.loads(str_raw_review)
            i_item += 1

            dict_review_payload = dict()
            for field in list_fields:  # write payload in one line
                dict_review_payload[field] = dict_review[field]
            file_output.write(json.dumps(dict_review_payload, ensure_ascii=False))
            file_output.write("\n")

            if i_item % 100 == 0:
                print("{} items got extracted payload.".format(i_item))
            if i_item == int_number_items:
                break
        except ValueError as e:
            pass

    file_input.close()
    file_output.close()


# инициализация файлов с данными
STR_PATH_FILE_DATA = abspath('../data/reviews_texts.txt')
STR_PATH_FILE_DATA_PAYLOAD = abspath('../data/payload/reviews_texts_1000.txt')

file_data = open(STR_PATH_FILE_DATA, 'r', encoding='utf-8')
file_data_payload = open(STR_PATH_FILE_DATA_PAYLOAD, 'w', encoding='utf-8')

# извлечение полезной информации из первых 500 отзывов
list_fields_payload = [
    'rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5',
    'recom_number', 'plus', 'minus', 'recom_author_mark',
    'title', 'overall', 'description'
]
extract_payload(file_data, file_data_payload, list_fields_payload, 1000)

100 items got extracted payload.
200 items got extracted payload.
300 items got extracted payload.
400 items got extracted payload.
500 items got extracted payload.
600 items got extracted payload.
700 items got extracted payload.
800 items got extracted payload.
900 items got extracted payload.
1000 items got extracted payload.


In [11]:
from pymystem3 import Mystem
from nltk.corpus import stopwords

# нормализация текста
mystem = Mystem()
list_stopwords = stopwords.words("russian")


def normalize_text(str_text):
    import re
    # лемматизация
    list_str_tokens = mystem.lemmatize(str_text.lower())
    # удаление стоп-слов, пунктуации
    list_str_tokens = [str_token for str_token in list_str_tokens if
                       str_token not in list_stopwords and
                       re.match('([а-яА-Я]+)(-?[а-яА-Я]+)', str_token)]

    return " ".join(list_str_tokens)


# нормализация полезной информации
def normalize(file_input, file_output, int_number_items: int) -> None:
    list_fields_rating = ['rating_{}'.format(i) for i in range(1, 6)]

    i_item = 0  # number of loaded items
    for str_raw_payload in file_input:
        dict_payload = json.loads(str_raw_payload)
        i_item += 1

        int_count_ratings = 0
        int_sum_rating = 0
        str_content = ""
        dict_payload_normalized = dict()
        for field in dict_payload.keys():  # по всем полям объекта
            if field in list_fields_rating and dict_payload[field]:     # поля рейтинга
                match = re.search('width: (\d+)%;', dict_payload[field])
                int_sum_rating += int(match.group(1))
                int_count_ratings += 1
            else:                                                       # текстовые поля с описанием
                str_content += " " + dict_payload[field]
        dict_payload_normalized["avg_rating"] = int_sum_rating // int_count_ratings
        dict_payload_normalized["content"] = normalize_text(str_content)

        file_output.write(json.dumps(dict_payload_normalized, ensure_ascii=False))
        file_output.write("\n")

        if i_item % 100 == 0:
            print("{} items are normalized.".format(i_item))
        if i_item == int_number_items:
            break

    file_input.close()
    file_output.close()

# инициализация файлов с данными
STR_PATH_FILE_DATA_PAYLOAD = abspath('../data/payload/reviews_texts_1000.txt')
STR_PATH_FILE_DATA_NORMALIZED = abspath('../data/normalized/reviews_texts_1000.txt')

file_data_payload = open(STR_PATH_FILE_DATA_PAYLOAD, 'r', encoding='utf-8')
file_data_normalized = open(STR_PATH_FILE_DATA_NORMALIZED, 'w', encoding='utf-8')

# нормализация полезной информации
normalize(file_data_payload, file_data_normalized, 1000)



100 items are normalized.
200 items are normalized.
300 items are normalized.
400 items are normalized.
500 items are normalized.
600 items are normalized.
700 items are normalized.
800 items are normalized.
900 items are normalized.
1000 items are normalized.


#### Задача № 1.
Выделить из корпуса по 500 примеров на каждый класс, если классификация бинарная и по 200, если классов несколько.

In [5]:
import ast

POS_RATING = 1
NEG_RATING = 0

def read_data(corpus_path):
    with open(corpus_path, 'r', encoding='utf-8') as f:
        print("Reading normalized file...")

        reviews = []
        avg_ratings = []

        lines = f.readlines()
        lines = [x.strip() for x in lines]

        for line in lines:
            dictionary = ast.literal_eval(line)
            reviews.append(dictionary['content'])

            avg_rating = dictionary['avg_rating']
            avg_ratings.append(parse_label(avg_rating))

    return reviews, avg_ratings


def parse_label(avg_rating):
    if avg_rating <= 60:
        return NEG_RATING
    else:
        return POS_RATING

def split_reviews(reviews, avg_ratings, n=500):
    pos_reviews = []
    neg_reviews = []

    if len(reviews) < n or n == -1:
        n = len(reviews)

    for index, review in enumerate(reviews):
        if avg_ratings[index] == POS_RATING and len(pos_reviews) < n:
            
            pos_reviews.append(review)
        elif avg_ratings[index] == NEG_RATING and len(neg_reviews) < n:
            neg_reviews.append(review)

        if len(pos_reviews) == n and len(neg_reviews) == n:
            break

    return pos_reviews, neg_reviews

def load_normalized_reviews(file_input, int_number_items:int) -> list:
    list_normalized_reviews = []
    int_i = 0
    for str_normalized_review in file_input:
        dict_normalized_review = json.loads(str_normalized_review)
        list_normalized_reviews.append((dict_normalized_review['content'], dict_normalized_review['avg_rating']))
        int_i += 1

        if int_i == int_number_items:
            break
    return list_normalized_reviews


file_data_normalized = open(STR_PATH_FILE_DATA_NORMALIZED, 'r', encoding='utf-8')
reviews, avg_ratings = read_data(STR_PATH_FILE_DATA_NORMALIZED)
pos_reviews, neg_reviews = split_reviews(reviews, avg_ratings)

y_pos = [1] * len(pos_reviews)
y_neg = [0] * len(neg_reviews)


#### Задача № 2.
Разбить примеры на обучающую и тестовую выборки в соотношении 80% к 20% соответственно.

In [13]:
from numpy.random import shuffle
from sklearn.model_selection import train_test_split

X_train_review, X_test_review, y_train_rating, y_test_rating = train_test_split(pos_reviews + neg_reviews, y_pos + y_neg, test_size=0.2)


In [16]:
# 3. Реализовать классификатор на основе tf-idf и модели логистической регрессии, вывести метрики качества на тестовом множестве. Максимальное кол-во признаков: 50000, минимальная частота: 5.
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support, classification_report
from pymystem3 import Mystem
from nltk.corpus import stopwords

def logistic_regression_fit(x_train, y_train, x_test, y_test):
    model = LogisticRegression(solver="lbfgs")
    model.fit(x_train, y_train)
    result_score(model, x_test, y_test)
    
def result_score(model, X_test, y_test):
    y_pred = model.predict(X_test)
    metrics = precision_recall_fscore_support(y_pred=y_pred, y_true=y_test, average='binary', pos_label=POS_RATING)
    print(f'Precision: {np.round(metrics[0], 3)}, recall: {np.round(metrics[1], 3)}, f-measure: {np.round(metrics[2], 3)}')
    print(classification_report(y_test, y_pred)) 

def tokenize(document):
    ignore = set(stopwords.words('russian'))
    stem = Mystem()

    tokens = stem.lemmatize(document)

    tokens = [w.lower() for w in tokens if w not in ignore]
    tokens = [w for w in tokens if w not in string.punctuation]
    tokens = [w for w in tokens if w.isalpha()]

    return tokens

vectorizer = TfidfVectorizer(max_features=50000, min_df=5, tokenizer=tokenize)
X_train_vect = vectorizer.fit_transform(X_train_review)
print(X_train_vect)
X_test_vect = vectorizer.transform(X_test_review)
print(X_test_review)

logistic_regression_fit(X_train_vect.toarray(), y_train_rating, X_test_vect.toarray(), y_test_rating)


  (0, 1279)	0.25323281543008586
  (0, 795)	0.14194652891201243
  (0, 1021)	0.2554062889151279
  (0, 720)	0.11900894539730555
  (0, 890)	0.03963387562390429
  (0, 1126)	0.06317404073680552
  (0, 427)	0.2905957318243978
  (0, 301)	0.1248194092594586
  (0, 162)	0.08342003997465011
  (0, 1507)	0.12085271391100097
  (0, 225)	0.08441093847669529
  (0, 1018)	0.1751727637763905
  (0, 121)	0.1751727637763905
  (0, 524)	0.14770065463301768
  (0, 111)	0.14194652891201243
  (0, 496)	0.12148686944986582
  (0, 490)	0.16255120820527025
  (0, 266)	0.2026448729197633
  (0, 796)	0.09564601423262034
  (0, 254)	0.10098598910451953
  (0, 825)	0.12085271391100097
  (0, 1061)	0.13986005372667323
  (0, 493)	0.18340231653463793
  (0, 1630)	0.12148686944986582
  (0, 1681)	0.1938008291606996
  :	:
  (487, 976)	0.18349114799813837
  (487, 1447)	0.10098399010678402
  (487, 487)	0.09562331539396841
  (487, 801)	0.12756246511062122
  (487, 1479)	0.25078694152727127
  (487, 653)	0.129920296202289
  (487, 119)	0.17670

c:\users\kurna\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# 4. Вывести наиболее значимые признаки (токены), используя один из указанных методов [3, 4]
from sklearn.ensemble import ExtraTreesClassifier

def feature_importances(X, Y):
    model = ExtraTreesClassifier()
    model.fit(X, Y)
    return model.feature_importances_


def print_features(tokens_importances, n=100):
    tokens_importances = zip(*sorted(zip(tokens_importances), reverse=True))
    print(tokens_importances[:n])

print('Features importance:')
token_importances = feature_importances(X_train_vect.toarray(), y_train_rating)
print_features(token_importances)


In [ ]:
# 5. Реализовать не менее 5-ти собственных признаков (можно больше), улучшающих результаты классификатора, полученные с использованием признака tf-idf. Оценить поочередно качество классификатора, обученного на tf-idf и каждом из признаков [5].

# Признак char ngram
import numpy as np

def get_char_ngram_feature(x_train, x_test):
    vectorizer = TfidfVectorizer(max_features=50000, min_df=5, analyzer='char', ngram_range=(3, 3))
    X_train_vect_char_ngram = vectorizer.fit_transform(x_train)
    X_test_vect_char_ngram = vectorizer.transform(x_test)

    return X_train_vect_char_ngram, X_test_vect_char_ngram


print('Features char ngram')
X_train_vect_char_ngram, X_test_vect_char_ngram = get_char_ngram_feature(X_train_review, X_test_review)

train_matrix = np.append(X_train_vect.toarray(), X_train_vect_char_ngram.toarray(), axis=1)
test_matrix = np.append(X_test_vect.toarray(), X_test_vect_char_ngram.toarray(), axis=1)

logistic_regression_fit(train_matrix, y_train_rating, test_matrix, y_test_rating)

In [ ]:
# Признак word ngram

def get_word_ngram_feature(x_train, x_test):
    vectorizer = TfidfVectorizer(max_features=50000, min_df=5, analyzer='word', ngram_range=(3, 3))
    X_train_vect_word_ngram = vectorizer.fit_transform(x_train)
    X_test_vect_word_ngram = vectorizer.transform(x_test)

    return X_train_vect_word_ngram, X_test_vect_word_ngram


print('Features word ngram')
X_train_vect_word_ngram, X_test_vect_word_ngram = get_word_ngram_feature(X_train_review, X_test_review)

train_matrix = np.append(X_train_vect.toarray(), X_train_vect_word_ngram.toarray(), axis=1)
test_matrix = np.append(X_test_vect.toarray(), X_test_vect_word_ngram.toarray(), axis=1)

logistic_regression_fit(train_matrix, y_train_rating, test_matrix, y_test_rating)


In [ ]:
# Признак brackets count

def get_bracket_count_feature(X_train, X_test):
    train_bracket_counts = [document.count(')') - document.count('(') for document in X_train]
    train_bracket_counts = np.reshape(train_bracket_counts, (-1, 1))

    test_bracket_counts = [document.count(')') - document.count('(') for document in X_test]
    test_bracket_counts = np.reshape(test_bracket_counts, (-1, 1))

    return train_bracket_counts, test_bracket_counts

print('Feature brackets count')
train_bracket_counts, test_bracket_counts = get_bracket_count_feature(X_train_review, X_test_review)

train_matrix = np.append(X_train_vect.toarray(), train_bracket_counts, axis=1)
test_matrix = np.append(X_test_vect.toarray(), test_bracket_counts, axis=1)

logistic_regression_fit(train_matrix, y_train_rating, test_matrix, y_test_rating)


In [ ]:
# 6. Оценить качество классификатора, обученного на tf-idf и всех реализованных признаках.

# оценки качества классификатора
#char n gram
train_matrix = np.append(X_train_vect.toarray(), X_train_vect_char_ngram.toarray(), axis=1)
test_matrix = np.append(X_test_vect.toarray(), X_test_vect_char_ngram.toarray(), axis=1)
#word ngram
train_matrix = np.append(train_matrix, X_train_vect_word_ngram.toarray(), axis=1)
test_matrix = np.append(test_matrix, X_test_vect_word_ngram.toarray(), axis=1)
#bracket counts
train_matrix = np.append(train_matrix, train_bracket_counts, axis=1)
test_matrix = np.append(test_matrix, test_bracket_counts, axis=1)

logistic_regression_fit(train_matrix, y_train_rating, test_matrix, y_test_rating)


In [ ]:
# 7. С помощью GridSearch вывести наиболее значимые признаки [4].
from sklearn.model_selection import GridSearchCV

def feature_importances_gridsearch(X, Y):
    model = GridSearchCV(estimator=ExtraTreesClassifier(), param_grid={})
    model.fit(X, Y)
    return model.best_estimator_.feature_importances_

print('Использование GridSearch для вывода признаков:')
token_importances = feature_importances_gridsearch(train_matrix, y_train_rating)
print_features(token_importances)

